In [4]:
import pandas as pd
import sys
import subprocess

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Ton fichier source (brut téléchargé depuis IHME)
INPUT_FILE = "IHME_Mental_Health_2021.csv"

# Le fichier de sortie (Tableau restructuré)
OUTPUT_FILE = "IHME_Mental_Health_WIDE.csv"

# ---------------------------------------------------------
# FONCTION DE PIVOT
# ---------------------------------------------------------
def restructure_data(input_path):
    print(f"1. Chargement du fichier : {input_path}")
    df = pd.read_csv(input_path)
    
    # Vérification rapide des colonnes
    print(f"   Colonnes trouvées : {list(df.columns)}")
    
    # On ne garde que les 3 colonnes essentielles pour le pivot
    # 'val' contient le chiffre (DALYs), 'cause_name' la maladie, 'location_name' le pays
    df_reduced = df[['location', 'cause', 'val']]
    
    print("2. Restructuration (Pivot) en cours...")
    # C'est ici que la magie opère :
    # - index='location_name' : Chaque pays devient une ligne unique
    # - columns='cause_name' : Chaque maladie devient une colonne
    # - values='val' : On remplit les cases avec le chiffre DALYs
    df_wide = df_reduced.pivot(index='location', columns='cause', values='val')
    
    # Après un pivot, l'index est complexe, on le remet à plat pour avoir une colonne 'Pays' normale
    df_wide = df_wide.reset_index()
    
    # 3. Nettoyage des noms de colonnes (pour éviter les espaces gênants dans les formules)
    # Ex: "Anxiety disorders" deviendra "Anxiety_disorders"
    print("3. Nettoyage des noms de colonnes...")
    df_wide.columns = [
        str(col).replace(' ', '_')      # Remplace espace par _
                .replace('-', '_')      # Remplace tiret par _
                .replace('/', '_')      # Remplace slash par _
                .replace('(', '')       # Enlève parenthèse ouvrante
                .replace(')', '')       # Enlève parenthèse fermante
                .replace("'", "")       # Enlève apostrophe
        for col in df_wide.columns
    ]
    
    # On renomme la colonne pays pour que ce soit clair
    df_wide = df_wide.rename(columns={'location_name': 'Country'})
    
    return df_wide

# ---------------------------------------------------------
# MAIN
# ---------------------------------------------------------
if __name__ == "__main__":
    try:
        # Exécution de la restructuration
        df_clean = restructure_data(INPUT_FILE)
        
        # Sauvegarde
        df_clean.to_csv(OUTPUT_FILE, index=False)
        
        print("\n" + "="*40)
        print("SUCCÈS ! TABLEAU RESTRUCTURÉ")
        print("="*40)
        print(f"Fichier sauvegardé ici : {OUTPUT_FILE}")
        print(f"Nombre de lignes (Pays) : {len(df_clean)}")
        print(f"Nombre de colonnes (Maladies) : {len(df_clean.columns)}")
        print("-" * 40)
        print("Aperçu des nouvelles colonnes :")
        print(df_clean.columns.tolist())
        
    except FileNotFoundError:
        print(f"ERREUR : Impossible de trouver le fichier {INPUT_FILE}")
        print("Vérifie que tu as bien mis le fichier téléchargé dans le dossier Données_OMS.")
    except KeyError as e:
        print(f"ERREUR : Colonne manquante {e}.")
        print("Vérifie que ton fichier contient bien 'location_name', 'cause_name' et 'val'.")

1. Chargement du fichier : IHME_Mental_Health_2021.csv
   Colonnes trouvées : ['measure', 'location', 'sex', 'age', 'cause', 'metric', 'year', 'val', 'upper', 'lower']
2. Restructuration (Pivot) en cours...
3. Nettoyage des noms de colonnes...

SUCCÈS ! TABLEAU RESTRUCTURÉ
Fichier sauvegardé ici : IHME_Mental_Health_WIDE.csv
Nombre de lignes (Pays) : 204
Nombre de colonnes (Maladies) : 12
----------------------------------------
Aperçu des nouvelles colonnes :
['location', 'Anxiety_disorders', 'Attention_deficit_hyperactivity_disorder', 'Autism_spectrum_disorders', 'Bipolar_disorder', 'Conduct_disorder', 'Depressive_disorders', 'Eating_disorders', 'Idiopathic_developmental_intellectual_disability', 'Mental_disorders', 'Other_mental_disorders', 'Schizophrenia']
